In [1]:
import tensorflow as tf

import numpy as np
import time


In [2]:
from names import check, raw_data, vectorize, seq_model
from names.rules_based import census, from_file, npc_gen

### Read the data

First, look in the text:

In [3]:
vocab = check.default_vocab()

In [4]:
df_census = census.census()
df_census

,name,gender,count,year,source
0,Mary,F,7065,1880,US census
1,Anna,F,2604,1880,US census
2,Emma,F,2003,1880,US census
3,Elizabeth,F,1939,1880,US census
4,Minnie,F,1746,1880,US census
...,...,...,...,...,...
31266,Zykell,M,5,2020,US census
31267,Zylus,M,5,2020,US census
31268,Zymari,M,5,2020,US census
31269,Zyn,M,5,2020,US census


In [5]:
df_jrpg = from_file.jrpg()
df_jrpg

,name,source
0,Adray Lasbard,jrpg names
1,Albel Nox,jrpg names
2,Cliff Fittir,jrpg names
3,Fayt Leingod,jrpg names
4,Maria Traydor,jrpg names
...,...,...
6450,Zurvan,jrpg names
6451,Zuta,jrpg names
6452,ZuZu,jrpg names
6453,Zyle,jrpg names


In [6]:
all_dfs = npc_gen.all_dfs()

In [7]:
all_dfs.append(df_census)
all_dfs.append(df_jrpg)

In [8]:
raw = []
for _ in range(100):
    raw = raw + raw_data.sample_from_dfs(all_dfs, k=100)

In [9]:
raw[-20:]

["{'source': 'jrpg names'}\nRagelise B Baloumat",
 "{'source': 'jrpg names'}\nMary",
 "{'source': 'jrpg names'}\nJovan",
 "{'source': 'jrpg names'}\nGarret",
 "{'source': 'jrpg names'}\nKulvan's Daughter",
 "{'source': 'jrpg names'}\nReks",
 "{'source': 'jrpg names'}\nZorn and Thorn",
 "{'source': 'jrpg names'}\nBroken Mountain",
 "{'source': 'jrpg names'}\nNótt",
 "{'source': 'jrpg names'}\nAsura",
 "{'source': 'jrpg names'}\nZangan",
 "{'source': 'jrpg names'}\nMaroda",
 "{'source': 'jrpg names'}\nKotokaze Benitoki",
 "{'source': 'jrpg names'}\nVelouria",
 "{'source': 'jrpg names'}\nBorlov",
 "{'source': 'jrpg names'}\nFayt Leingod",
 "{'source': 'jrpg names'}\nTigris",
 "{'source': 'jrpg names'}\nGrog",
 "{'source': 'jrpg names'}\nVelibor Azetina",
 "{'source': 'jrpg names'}\nMaria"]

In [10]:
text = raw_data.join_raw(raw)

In [11]:
text[:300]

"{'gender': 'F', 'source': 'npc female dwarf'}\nThraa                                                 \n{'gender': 'F', 'source': 'npc female dwarf'}\nEbloda                                                \n{'gender': 'F', 'source': 'npc female dwarf'}\nGunnrasa                                            "

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [12]:
ds = vectorize.DataSet(text, vocab=vocab)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [13]:
vectorize.split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [14]:
# Length of the vocabulary in chars
# vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = [1024, 1024]

In [15]:
model = seq_model.SeqModel(vocab,
        embedding_dim=embedding_dim,
        rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [16]:
for input_example_batch, target_example_batch in ds.dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 179) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [17]:
model.summary()

Model: "seq_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  45824     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
gru_1 (GRU)                  multiple                  6297600   
_________________________________________________________________
dense (Dense)                multiple                  183475    
Total params: 10,465,203
Trainable params: 10,465,203
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [19]:
sampled_indices

array([ 24, 146,  50, 153, 154,  20, 161,  16,  79, 157, 104, 104,  30,
        58, 146,  60, 174,  41,  42,  32,  64,  42,   0,  54,  96, 143,
        80,  41, 112, 151,  97,   8, 127,  91, 109,   2, 174, 171,  65,
       160,  62,  22, 131,  41, 146,   0,  16,  44,   8,  55,  64,  90,
       137, 173,  95,  33,  21, 119,  95,  70, 142, 120, 107,  51, 140,
        85, 162, 157, 118,  58, 151,  76,  33, 132, 167,  79, 112,  69,
       155,  98,  72,  70, 133,  76, 172,   4,   1, 169,  15,  20,  36,
       117, 124, 140,  88, 165, 163,  79, 147,  65], dtype=int64)

Decode these to see the text predicted by this untrained model:

In [20]:
print("Input:\n", ds.text_from_ids(np.array([input_example_batch[0]])))
print()
print("Next Char Predictions:\n", ds.text_from_ids(np.array([sampled_indices])))

Input:
 ["{'gender': 'F', 'source': 'npc female human'}\nZhanna                                                "]

Next Char Predictions:
 ['-γἔ\\"ą,ὰrømmáñγ\t/pίώτί[UNK]ἰyjcpë!íkłùRv/|Â ldàpγ[UNK]ὰak<τπï%υL2Wυu#äε}èU0øSñ!ÕLAαrë]\n\'δuΩÕÓ9$ῦ+ąCÄâèübÖrςÂ']


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [21]:
example_batch_loss = model.loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 179)  # (batch_size, sequence_length, vocab_size)
Mean loss:         5.1878495


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [22]:
tf.exp(mean_loss).numpy()

179.08302

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [23]:
model.compile_model()

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [24]:
chk = check.CheckPoint("all_df_model")

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [25]:
EPOCHS = 20

In [26]:
history = model.fit_with_chk(ds, checkpoint=chk, epochs=EPOCHS)

Epoch 1/20
1365/1365 [==============================] - 125s 89ms/step - loss: 0.3211
Epoch 2/20
1365/1365 [==============================] - 126s 92ms/step - loss: 0.1727
Epoch 3/20
1365/1365 [==============================] - 127s 93ms/step - loss: 0.1549
Epoch 4/20
1365/1365 [==============================] - 127s 92ms/step - loss: 0.1401
Epoch 5/20
1365/1365 [==============================] - 127s 92ms/step - loss: 0.1320
Epoch 6/20
1365/1365 [==============================] - 126s 92ms/step - loss: 0.1266
Epoch 7/20
1365/1365 [==============================] - 126s 92ms/step - loss: 0.1228
Epoch 8/20
1365/1365 [==============================] - 126s 92ms/step - loss: 0.1190
Epoch 9/20
1365/1365 [==============================] - 126s 92ms/step - loss: 0.1159
Epoch 10/20
1365/1365 [==============================] - 126s 92ms/step - loss: 0.1151
Epoch 11/20
1365/1365 [==============================] - 126s 92ms/step - loss: 0.1149
Epoch 12/20
1365/1365 [=============================

In [27]:
one_step_model = seq_model.OneStep(model)

In [28]:
chk.save(model)

In [29]:
def generate_names(stem="", n=1):
    names = []
    for _ in range(n):
        states = None
        next_char = [stem + ch for ch in "QWERTYUIOPASDFGHJKLZXCVBNM"]
        next_char = tf.constant(next_char)
        result = [next_char]

        for n in range(40):
          next_char, states = one_step_model.generate_one_step(next_char, states=states)
          result.append(next_char)

        result = tf.strings.join(result)
        names += [" ".join([a.strip() for a in name.decode("utf-8").split() if len(a)]) for name in result.numpy()]
    return names

In [30]:
generate_names(stem="{'source': 'jrpg names'}\n", n=10)

["{'source': 'jrpg names'} Queen Seline",
 "{'source': 'jrpg names'} Waimemant Soribo",
 "{'source': 'jrpg names'} Elifa",
 "{'source': 'jrpg names'} Rowmen",
 "{'source': 'jrpg names'} T'yanch",
 "{'source': 'jrpg names'} Yue",
 "{'source': 'jrpg names'} Ulark",
 "{'source': 'jrpg names'} Ingus",
 "{'source': 'jrpg names'} Olvara",
 "{'source': 'jrpg names'} Pryna",
 "{'source': 'jrpg names'} Athello",
 "{'source': 'jrpg names'} Shoga",
 "{'source': 'jrpg names'} Duke Vox",
 "{'source': 'jrpg names'} Fidel Camuze",
 "{'source': 'jrpg names'} Garill",
 "{'source': 'jrpg names'} Hob",
 "{'source': 'jrpg names'} Jantellot de Thelomaire",
 "{'source': 'jrpg names'} Kevin Bachtein",
 "{'source': 'jrpg names'} Loot",
 "{'source': 'jrpg names'} Zophiel",
 "{'source': 'jrpg names'} X'rhhun Tiint",
 "{'source': 'jrpg names'} Cidouse",
 "{'source': 'jrpg names'} Vayne's Reurstinel",
 "{'source': 'jrpg names'} Bias",
 "{'source': 'jrpg names'} Nyle",
 "{'source': 'jrpg names'} Maroda",
 "{'sourc

In [31]:
generate_names("{'gender': 'F', 'source': 'npc female dwarf'}\n")

["{'gender': 'F', 'source': 'npc female dwarf'} Quari",
 "{'gender': 'F', 'source': 'npc female dwarf'} Wharla",
 "{'gender': 'F', 'source': 'npc female dwarf'} Ebloda",
 "{'gender': 'F', 'source': 'npc female dwarf'} Rurnal",
 "{'gender': 'F', 'source': 'npc female dwarf'} Toris",
 "{'gender': 'F', 'source': 'npc female dwarf'} Yuris",
 "{'gender': 'F', 'source': 'npc female dwarf'} Ulfis",
 "{'gender': 'F', 'source': 'npc female dwarf'} Iil",
 "{'gender': 'F', 'source': 'npc female dwarf'} Ohild",
 "{'gender': 'F', 'source': 'npc female dwarf'} Palnal",
 "{'gender': 'F', 'source': 'npc female dwarf'} Audloda",
 "{'gender': 'F', 'source': 'npc female dwarf'} Soril",
 "{'gender': 'F', 'source': 'npc female dwarf'} Dkil",
 "{'gender': 'F', 'source': 'npc female dwarf'} Filif",
 "{'gender': 'F', 'source': 'npc female dwarf'} Golloda",
 "{'gender': 'F', 'source': 'npc female dwarf'} Hlla",
 "{'gender': 'F', 'source': 'npc female dwarf'} Jilloda",
 "{'gender': 'F', 'source': 'npc female dw

In [32]:
generate_names("{'gender': 'M', 'source': 'npc male dwarf'}\n")

["{'gender': 'M', 'source': 'npc male dwarf'} Queen",
 "{'gender': 'M', 'source': 'npc male dwarf'} Wharik",
 "{'gender': 'M', 'source': 'npc male dwarf'} Ebik",
 "{'gender': 'M', 'source': 'npc male dwarf'} Rurur",
 "{'gender': 'M', 'source': 'npc male dwarf'} Thraik",
 "{'gender': 'M', 'source': 'npc male dwarf'} Yurur",
 "{'gender': 'M', 'source': 'npc male dwarf'} Ulfar",
 "{'gender': 'M', 'source': 'npc male dwarf'} Iblud",
 "{'gender': 'M', 'source': 'npc male dwarf'} Obon",
 "{'gender': 'M', 'source': 'npc male dwarf'} Polin",
 "{'gender': 'M', 'source': 'npc male dwarf'} Artbon",
 "{'gender': 'M', 'source': 'npc male dwarf'} Sonic",
 "{'gender': 'M', 'source': 'npc male dwarf'} Dwalerk",
 "{'gender': 'M', 'source': 'npc male dwarf'} Filic",
 "{'gender': 'M', 'source': 'npc male dwarf'} Gunnto",
 "{'gender': 'M', 'source': 'npc male dwarf'} Hlerk",
 "{'gender': 'M', 'source': 'npc male dwarf'} Jilik",
 "{'gender': 'M', 'source': 'npc male dwarf'} Korlinn",
 "{'gender': 'M', 'sou

In [33]:
generate_names("{'source': 'jrpg names'}\nJoby the ")

["{'source': 'jrpg names'} Joby the Quore",
 "{'source': 'jrpg names'} Joby the Wale",
 "{'source': 'jrpg names'} Joby the Eston",
 "{'source': 'jrpg names'} Joby the Rogan",
 "{'source': 'jrpg names'} Joby the Thomen of Stwille",
 "{'source': 'jrpg names'} Joby the Youngeles",
 "{'source': 'jrpg names'} Joby the Ulgid",
 "{'source': 'jrpg names'} Joby the Indolstak",
 "{'source': 'jrpg names'} Joby the Ornorie",
 "{'source': 'jrpg names'} Joby the Proekser",
 "{'source': 'jrpg names'} Joby the Armainbeldt",
 "{'source': 'jrpg names'} Joby the Shonnel",
 "{'source': 'jrpg names'} Joby the Delmatte",
 "{'source': 'jrpg names'} Joby the Forney",
 "{'source': 'jrpg names'} Joby the Gunters",
 "{'source': 'jrpg names'} Joby the Hext",
 "{'source': 'jrpg names'} Joby the Jatore",
 "{'source': 'jrpg names'} Joby the Kentoo",
 "{'source': 'jrpg names'} Joby the Lohe",
 "{'source': 'jrpg names'} Joby the Zeola",
 "{'source': 'jrpg names'} Joby the XInano",
 "{'source': 'jrpg names'} Joby the C

In [34]:
generate_names(stem="{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'}\n")

["{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Quinn",
 "{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Wedelyne",
 "{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Estee st",
 "{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Roxana vi",
 "{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Themeca",
 "{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Yodel",
 "{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Ulgie",
 "{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Irene ly",
 "{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Oyachume e",
 "{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Peyton chi",
 "{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Anna",
 "{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Sonna",
 "{'gender': 'F', 'count': 7065, 'year':

In [35]:
generate_names(stem="{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'}\n")

["{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Queen ie",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Weston e",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Eller es",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Rollie",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Tiey",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Yabor",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Ulis",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Ilee",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Odell",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Philope",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Avontae",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Sullivan",
 "{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'} Dellio

In [36]:
raw_data.row2sample(df_census.iloc[0])

"{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'}\nMary"

In [37]:
place = []
for name in generate_names():
    name = name.split()[0]
    for long_name in generate_names(name + " of "):
        print(long_name)
        place.append(long_name.split()[2])

QuLShark of Qegre wah
QuLShark of Warns ish
QuLShark of Els
QuLShark of Raly
QuLShark of Thalmarc
QuLShark of Ya us
QuLShark of UShancant is a enthis
QuLShark of I wit
QuLShark of Oman
QuLShark of Palhandem: ':mances': 'sour':': 's urce':
QuLShark of Ald Mat
QuLShark of Shaminder': 'sour': ': 'sour': ': 'r'ur':
QuLShark of DallsPi:t's'8an': ': 'sour': ': cen'ur':'
QuLShark of Farcha vil
QuLShark of Galvi
QuLShark of Hl0:
QuLShark of Jalla-PKfidras is
QuLShark of Kanna
QuLShark of Laphari: vin
QuLShark of Zalmas
QuLShark of Xa el
QuLShark of Car': sied
QuLShark of Vall us ': as uce':
QuLShark of Bardi:
QuLShark of Nan is
QuLShark of Mangel chi
WhwSitha of QiXS1mannus': 'soune': 'US census'} Satar
WhwSitha of Wim: 'source': 's urce': 'US census'} Mar
WhwSitha of Ensemar': 'source': 'source': 'source': '
WhwSitha of Rikb: 5sour ': ': 'sourc:': ': gnor': 'F'
WhwSitha of T'anger': 's ur': 's urce': 's'S 'sus'} O
WhwSitha of Ysevi: ': 'sour': ': 's urce': 'US census
WhwSitha of UMmar':r':': 

And'ur': of Quuumen'} Sammat
And'ur': of W'alc 'source': 'US census'} Harrell
And'ur': of Er': 'M', 'source': 'US census'} Emers
And'ur': of Ruurc': 'n c 's'1um: 'source': 'US census
And'ur': of T'orce': 'US census'} Jadriel
And'ur': of Y'urce': 'M', 'source': 'US census'} Layt
And'ur': of U: 1 '1 'n ce's'} Percil
And'ur': of I'g'n ce':': 193 ': 1987, 'source': 'US c
And'ur': of O'' ': 'n'c 's'uuc ': 'lint'gendes'} Mill
And'ur': of P'','ce's 'umc ch
And'ur': of Ar': 'M', 'counc': 112, 'year': 1985, 'so
And'ur': of S'c 'n'c 'n'c 'amc 'npc iame '} 7lia
And'ur': of DF us'} Duri
And'ur': of F'gen'e ': '8, 'source': 'US census'} Mar
And'ur': of Gur': 'US census'} Pranwes
And'ur': of He'g'} Ruric
And'ur': of Jer'c ': 1946, 'source': 'US census'} Old
And'ur': of Kur': 'n'c 'a ': 1951, 'source': 'US cens
And'ur': of Lo 'n'g 'nog ': 'npe malg dwarf'} Audlinn
And'ur': of Zuur':': 'n0c 'nmc 'emc 'n cemale 'M', 's
And'ur': of XF 'ome ': ce's'} Prillie
And'ur': of Cume': 'npc 'ale dwarf'} Ballum
An

XVV8m:t'source': of Q Su8si:
XVV8m:t'source': of W erce': 'n1pece': c 1iche ce's 8icthandw
XVV8m:t'source': of Er': cec'n 11676 , ssust'} Lynder
XVV8m:t'source': of R uscesic 'sisrasiamerkalesel: 's'urce':
XVV8m:t'source': of T'yri: 's 19: 'e ce': 1918, 'source': 'US
XVV8m:t'source': of Yrce': cece's'u census':
XVV8m:t'source': of U: cene'} Dia
XVV8m:t'source': of Ir':': 's'T ami
XVV8m:t'source': of Or': er': 'US census'} Jaccour
XVV8m:t'source': of Pr16: cec'e 's11nce's'} Tenis
XVV8m:t'source': of Arce': er': 's cinder
XVV8m:t'source': of S'cece':'} Artan
XVV8m:t'source': of D: c cem:'} Thorar
XVV8m:t'source': of Fr'a'c ': ceces'} Alianita
XVV8m:t'source': of Grurac': 1695, 'source': 'US census'} Dia
XVV8m:t'source': of He's urces'} Daulton
XVV8m:t'source': of Jeranche i
XVV8m:t'source': of Kerce': nnoge': 9Amer': 19c6, 'source':
XVV8m:t'source': of L: 1nnece': ceces'} Balder
XVV8m:t'source': of Z ercanses': 9:r'ncas': 'sour':': 'd c ':
XVV8m:t'source': of X:11epe'} Dwanic
XVV8m:t'sou

In [38]:
place

['Qegre',
 'Warns',
 'Els',
 'Raly',
 'Thalmarc',
 'Ya',
 'UShancant',
 'I',
 'Oman',
 'Palhandem:',
 'Ald',
 "Shaminder':",
 "DallsPi:t's'8an':",
 'Farcha',
 'Galvi',
 'Hl0:',
 'Jalla-PKfidras',
 'Kanna',
 'Laphari:',
 'Zalmas',
 'Xa',
 "Car':",
 'Vall',
 'Bardi:',
 'Nan',
 'Mangel',
 "QiXS1mannus':",
 'Wim:',
 "Ensemar':",
 'Rikb:',
 "T'anger':",
 'Ysevi:',
 "UMmar':r':':",
 "Ince's",
 "Onder':",
 "Peur':",
 "Asrckar':",
 "Sman':",
 'Dami:',
 "Fleuer':",
 "Girr':",
 'HlJumi:',
 'Jenstria',
 "Keles':",
 'LX0119:',
 "Zerker':",
 'XS',
 "Crancust':rus",
 'Valinus',
 'Berm:l:',
 "Neur':",
 "Mi:ces':",
 'Quimi',
 'Wis',
 'Elko',
 'Ruum',
 'T',
 'Yus',
 'UMoni:',
 'I',
 'O',
 'Pans',
 "As':",
 'S',
 'Daugh',
 'Finlang',
 'Ganleug',
 'Hiand',
 'Jiald',
 'KVimans',
 'Luuf',
 'Zumin',
 'XIV',
 'Cher',
 'Valluus',
 "Bunthernur':",
 'Nana',
 'Mansuma',
 'Qe11',
 'Wal:',
 "Est':",
 'R',
 'T',
 'Yang',
 "U:moroce':",
 'I',
 "O'snce':",
 "Pi:t'sor':",
 "Asker':",
 "S9:e'sourc':",
 'Dahumi:',
 'Fyn

In [39]:
G = list(filter(lambda name: name[0] == "G", place))
G.sort()
G

["G0org':",
 "GFring'}",
 'Ga',
 'Galle',
 'Gallene',
 'Gallere',
 'Galvi',
 "Ganger':",
 'Ganleug',
 'Garne',
 'Ge',
 'Genti:',
 'Gera',
 "Getry':",
 'Giand',
 'Gill',
 "Girr':",
 "Gorr':",
 'Gorv:',
 "Gronan':':",
 "Grurac':",
 "Gunn':",
 "Gur':",
 "Gura':",
 "Gurc':",
 'Guri:']

In [40]:
np.random.choice(place, size=10, replace=False)

array(['Naus', 'Zynal', "Mesas'Fer':", 'Jume', "Zouce':", "Mansuma':",
       "QuMorges'}", "Yong':", "N'sour's':", 'Daugelis'], dtype='<U26')

In [41]:
np.random.choice(place, size=10, replace=False)

array(["Ruurc':", 'I', 'I', 'Daugh', "DM',", "Pnnnus'}", "Es'urce':",
       'Lexer', 'LF', 'Reuc'], dtype='<U26')

In [42]:
np.random.choice(place, size=10, replace=False)

array(['Fris', "Mensor':", "O'g", "Z399:t'source':", "F'gender':",
       "Lelgenus'}", 'Valinus', 'Elko', 'Jenna', 'T'], dtype='<U26')

In [43]:
generate_names()

["Qu>hal samender': 'M'",
 'Wφym wel',
 'Es-uIng w',
 'RLumunchia',
 "T'oran",
 "Y'gennur': 'M', 'source': 'npc male dwarf",
 "U: nder': 'n8, 'source': 'npc male elf'}",
 "I'gender': uma 's'M aver",
 "O'gngne Der",
 "Pminourc:': ance': 1vint'yla dwarp: 'Mpc",
 "Ald': urce': 'nourc:': 1994, 'source': 'U",
 "S'eechura': 'F'' 'gene': 'source': 'jr5,",
 "Dw9rveneke S1ander': 'US census'} Ste",
 "F'gender': 'F', 'source': 'npc female dwa",
 'Gurva',
 "Hwynde':': 1880: 'source': 'US census'} H",
 'JJKy',
 'K. Scieper',
 "L,'source': 'source': 'source': 'US censu",
 'ZçGil v',
 "XV0, 'source': 'US census'} Gemma",
 'Cherle',
 'Viίxis er',
 "Bro': ':': 'eo'e': 'doge': 'US 'epge's'}",
 'Ncothal er',
 "M': 'source': 'npc male human'} Minthar"]